# EAS Testing - YouTube on Android

The goal of this experiment is to run Youtube videos on a Nexus N5X running Android with an EAS kernel and collect results. The Analysis phase will consist in comparing EAS with other schedulers, that is comparing *sched* governor with:

    - interactive
    - performance
    - powersave
    - ondemand

In [1]:
import logging
from conf import LisaLogging
LisaLogging.setup()

2016-12-06 19:46:08,105 INFO    : root         : Using LISA logging configuration:
2016-12-06 19:46:08,106 INFO    : root         :   /home/vagrant/lisa/logging.conf


In [2]:
%pylab inline

import os
from time import sleep

# Support to access the remote target
import devlib
from env import TestEnv

from devlib.utils.android import adb_command

# Import support for Android devices
from android import System

# Support for trace events analysis
from trace import Trace

# Suport for FTrace events parsing and visualization
import trappy

Populating the interactive namespace from numpy and matplotlib


# Test Environment set up

In case more than one Android device are conencted to the host, you must specify the ID of the device you want to target in `my_target_conf`. Run `adb devices` on your host to get the ID.

In [3]:
# Setup a target configuration
my_target_conf = {
    
    # Target platform and board
    "platform"    : 'android',

    # Add target support
    "board" : 'pixel',
    
    # Device ID
    "device" : "HT6670300102",
    
    # ANDROID_HOME
    "ANDROID_HOME"  : '/home/vagrant/lisa/tools/android-sdk-linux/',
    
    # Define devlib modules to load
    "modules"     : [
        'cpufreq'       # enable CPUFreq support
    ],
}

In [4]:
my_tests_conf = {

    # Folder where all the results will be collected
    "results_dir" : "Android_Youtube",

    # Platform configurations to test
    "confs" : [
        {
            "tag"            : "youtube",
            "flags"          : "ftrace",           # Enable FTrace events
            "sched_features" : "ENERGY_AWARE",     # enable EAS
        },
    ],
    
    # FTrace events to collect for all the tests configuration which have
    # the "ftrace" flag enabled
    "ftrace"  : {
         "events" : [
            "sched_switch",
            "sched_load_avg_cpu",
            "cpu_frequency",
            "cpu_capacity"
         ],
         "buffsize" : 10 * 1024,
    },
    
    # Tools required by the experiments
    "tools"   : [ 'trace-cmd' ],
}

In [5]:
# Ensure ADB has root priviledges, which are required by systrace
!adb root

adbd is already running as root


In [6]:
# Initialize a test environment using:
# the provided target configuration (my_target_conf)
# the provided test configuration   (my_test_conf)
te = TestEnv(target_conf=my_target_conf, test_conf=my_tests_conf)
target = te.target

2016-12-06 19:46:11,262 INFO    : TestEnv      : Using base path: /home/vagrant/lisa
2016-12-06 19:46:11,263 INFO    : TestEnv      : Loading custom (inline) target configuration
2016-12-06 19:46:11,264 INFO    : TestEnv      : Loading custom (inline) test configuration
2016-12-06 19:46:11,264 INFO    : TestEnv      : External tools using:
2016-12-06 19:46:11,264 INFO    : TestEnv      :    ANDROID_HOME: /home/vagrant/lisa/tools/android-sdk-linux/
2016-12-06 19:46:11,265 INFO    : TestEnv      :    CATAPULT_HOME: /home/vagrant/lisa/tools/catapult
2016-12-06 19:46:11,265 INFO    : TestEnv      : Loading board:
2016-12-06 19:46:11,266 INFO    : TestEnv      :    /home/vagrant/lisa/libs/utils/platforms/pixel.json
2016-12-06 19:46:11,266 INFO    : TestEnv      : Devlib modules to load: [u'bl', u'cpufreq']
2016-12-06 19:46:11,267 INFO    : TestEnv      : Connecting Android target [HT6670300102]
2016-12-06 19:46:11,267 INFO    : TestEnv      : Connection settings:
2016-12-06 19:46:11,267 INF

# Support Functions

This set of support functions will help us running the benchmark using different CPUFreq governors.

In [7]:
def set_performance():
    target.cpufreq.set_all_governors('performance')

def set_powersave():
    target.cpufreq.set_all_governors('powersave')

def set_interactive():
    target.cpufreq.set_all_governors('interactive')

def set_sched():
    target.cpufreq.set_all_governors('sched')

def set_ondemand():
    target.cpufreq.set_all_governors('ondemand')
    
    for cpu in target.list_online_cpus():
        tunables = target.cpufreq.get_governor_tunables(cpu)
        target.cpufreq.set_governor_tunables(
            cpu,
            'ondemand',
            **{'sampling_rate' : tunables['sampling_rate_min']}
        )

In [8]:
# CPUFreq configurations to test
confs = {
    'performance' : {
        'label' : 'prf',
        'set' :  set_performance,
    },
    #'powersave' : {
    #    'label' : 'pws',
    #    'set' :  set_powersave,
    #},
     'interactive' : {
         'label' : 'int',
         'set' :  set_interactive,
     },
    #'sched' : {
    #    'label' : 'sch',
    #    'set' :  set_sched,
    #},
    #'ondemand' : {
    #    'label' : 'odm',
    #    'set' :  set_ondemand,
    #}
}

# The set of results for each comparison test
results = {}

In [9]:
YOUTUBE_CMD = 'shell dumpsys gfxinfo com.google.android.youtube > {}'

def youtube_run(exp_dir, video_url, video_duration_s):
    # Unlock device screen (assume no password required)
    target.execute('input keyevent 82')
    # Press Back button to be sure we run the video from the start
    target.execute('input keyevent KEYCODE_BACK')

    # Start YouTube video on the target device
    target.execute('am start -a android.intent.action.VIEW "{}"'.format(video_url))
    # Allow the activity to start
    sleep(3)
    # Reset framestats collection
    target.execute('dumpsys gfxinfo --reset')
    # Wait until the end of the video
    sleep(video_duration_s)
    
    # Get frame stats
    framestats_file = os.path.join(exp_dir, "framestats.txt")
    adb_command(target.adb_name, YOUTUBE_CMD.format(framestats_file))

    # Close application
    target.execute('am force-stop com.google.android.youtube')

    # Clear application data
    target.execute('pm clear com.google.android.youtube')

    return framestats_file

In [10]:
def experiment(governor, exp_dir,  collect='ftrace', trace_time=30):
    os.system('mkdir -p {}'.format(exp_dir));

    logging.info('------------------------')
    logging.info('Run workload using %s governor', governor)
    confs[governor]['set']()

    # Start the required tracing command
    if 'ftrace' in collect:
        # Start FTrace and Energy monitoring
        te.ftrace.start()
    elif 'systrace' in collect:
        # Start systrace
        trace_file = os.path.join(exp_dir, 'trace.html')
        systrace_output = System.systrace_start(te, trace_file, trace_time)

    ### Run the benchmark ###
    framestats_file = youtube_run(exp_dir, "https://youtu.be/XSGBVzeBUbk?t=45s", trace_time)

    # Stop the required trace command
    if 'ftrace' in collect:
        te.ftrace.stop()
        # Collect and keep track of the trace
        trace_file = os.path.join(exp_dir, 'trace.dat')
        te.ftrace.get_trace(trace_file)
    elif 'systrace' in collect:
        if systrace_output:
            logging.info('Waiting systrace report [%s]...', trace_file)
            systrace_output.wait()
        else:
            logging.warning('Systrace is not running!') 

    # Parse trace
    tr = Trace(te.platform, trace_file,
               events=my_tests_conf['ftrace']['events'])

    # return all the experiment data
    return {
        'dir'             : exp_dir,
        'framestats_file' : framestats_file,
        'trace'           : trace_file,
        'ftrace'          : tr.ftrace
    }

# Run experiments and collect traces

In [11]:
# Run the benchmark in all the configured governors
for governor in confs:
    test_dir = os.path.join(te.res_dir, governor)
    results[governor] = experiment(governor, test_dir,
                                   collect='systrace', trace_time=15)

2016-12-06 19:46:21,457 INFO    : root         : ------------------------
2016-12-06 19:46:21,458 INFO    : root         : Run workload using performance governor
2016-12-06 19:46:21,556 INFO    : System       : SysTrace: /home/vagrant/lisa/tools/catapult/systrace/systrace/run_systrace.py -e HT6670300102 -o /home/vagrant/lisa/results/Android_Youtube/performance/trace.html gfx view sched freq idle -t 15
2016-12-06 19:46:42,226 INFO    : root         : Waiting systrace report [/home/vagrant/lisa/results/Android_Youtube/performance/trace.html]...
2016-12-06 19:46:43,203 INFO    : Trace        : Parsing SysTrace format...
2016-12-06 19:46:48,092 INFO    : Trace        : Collected events spans a 9.888 [s] time interval
2016-12-06 19:46:48,092 INFO    : Trace        : Set plots time range to (0.000000, 9.888454)[s]
2016-12-06 19:46:48,093 INFO    : Analysis     : Registering trace analysis modules:
2016-12-06 19:46:48,094 INFO    : Analysis     :    tasks
2016-12-06 19:46:48,094 INFO    : An

# UI Performance Analysis

In [12]:
for governor in confs:
    framestats_file = results[governor]['framestats_file']
    print "Frame Statistics for {} governor".format(governor.upper())
    !sed '/Stats since/,/99th/!d;/99th/q' $framestats_file
    print ""

Frame Statistics for PERFORMANCE governor
Stats since: 107266916813060ns
Total frames rendered: 747
Janky frames: 44 (5.89%)
50th percentile: 5ms
90th percentile: 8ms
95th percentile: 19ms
99th percentile: 113ms

Frame Statistics for INTERACTIVE governor
Stats since: 107266916813060ns
Total frames rendered: 942
Janky frames: 60 (6.37%)
50th percentile: 5ms
90th percentile: 9ms
95th percentile: 20ms
99th percentile: 113ms



In [13]:
trace_file = results['interactive']['trace']
!xdg-open {trace_file}